In [12]:
import numpy as np
import pandas as pd

# plot
import matplotlib.pyplot as plt
import urllib.request

# tf
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

## URL Dataset

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


url relative path ke dataset

In [14]:
url = '/content/drive/MyDrive/Dataset'

# Data Train & Validation

Disini saya menggunakan Image data generator dan batch size sebanyak 128
untuk validation, saya menggunakan 20% data.
Kemudian untuk subset training dan validation, target size nya adalah 120.

In [22]:
batch_size = 128

train_image_generator = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=40,
    horizontal_flip=True,
    )

train = train_image_generator.flow_from_directory(
    url, 
    target_size=(120,120),
    batch_size=batch_size, 
    subset='training'
    )

val = train_image_generator.flow_from_directory(
    url, 
    target_size=(120,120),
    batch_size=batch_size,
    subset='validation'
    )

Found 3561 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


# Model 

Pada syntax di atas model dibentuk dengan suatu layer convo 2 dimensi dari size 120 x 120 pixels dimana fungsi aktivasinya adalah. Terdapat dua pembuatan layer convo, sehingga dilanjutkan dengan pembuatan layer max pooling dan pembuatan layer dropout. Karena ini adalah pelatihan deep learning maka perlu pelatihan lagi dengan membuat lagi layer untuk pelatihan. Kemudian dilanjutkan dengan layer flatten dan layer dense sampai dengan proses kompilasi.



In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        activation='relu',
        input_shape=(120, 120, 3), 
        padding='same'
        ),

      tf.keras.layers.Conv2D(
        64, (3,3), 
        activation='relu',
        input_shape=(120, 120, 3),
        padding='same'
        ),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(
        256, 
        (3,3), 
        activation='relu', 
        padding='same'
        ),
    
    tf.keras.layers.Conv2D(
      256, 
      (3,3), 
      activation='relu',
      padding='same'
      ),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')     
])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])


In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 120, 120, 64)      1792      
                                                                 
 conv2d_15 (Conv2D)          (None, 120, 120, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 60, 60, 64)       0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 60, 60, 64)        0         
                                                                 
 conv2d_16 (Conv2D)          (None, 60, 60, 256)       147712    
                                                                 
 conv2d_17 (Conv2D)          (None, 60, 60, 256)       590080    
                                                      

In [28]:
steps, val_steps = train.n/batch_size, val.n/batch_size
num_epochs = 50

epoch dibuat sebanyak 50 

In [29]:
history = model.fit(train, validation_data=val, epochs=num_epochs,
                    steps_per_epoch=steps, validation_steps=val_steps)

Epoch 1/50
27/27 [==============================] - 35s 1s/step - loss: 1.7431 - accuracy: 0.2213 - val_loss: 1.5866 - val_accuracy: 0.3108
Epoch 2/50
27/27 [==============================] - 30s 1s/step - loss: 1.6006 - accuracy: 0.2421 - val_loss: 1.5796 - val_accuracy: 0.3198
Epoch 3/50
27/27 [==============================] - 31s 1s/step - loss: 1.5031 - accuracy: 0.3336 - val_loss: 1.5290 - val_accuracy: 0.3164
Epoch 4/50
27/27 [==============================] - 31s 1s/step - loss: 1.4056 - accuracy: 0.3797 - val_loss: 1.4759 - val_accuracy: 0.3514
Epoch 5/50
27/27 [==============================] - 32s 1s/step - loss: 1.3416 - accuracy: 0.4167 - val_loss: 1.4401 - val_accuracy: 0.3592
Epoch 6/50
27/27 [==============================] - 31s 1s/step - loss: 1.3002 - accuracy: 0.4358 - val_loss: 1.3366 - val_accuracy: 0.4651
Epoch 7/50
27/27 [==============================] - 31s 1s/step - loss: 1.2528 - accuracy: 0.4732 - val_loss: 1.3043 - val_accuracy: 0.4651
Epoch 8/50
27/27 [==

In [33]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Akurasi Training')
plt.plot(epochs, val_acc, 'g', label='Akurasi Validation')

plt.xlabel('Epochs')
plt.ylabel('Akurasi')
plt.title('Akurasi Training and validation')
plt.grid(axis='both')

plt.show() 